In [1]:
import os
import sys
import json
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import LLAMAconnect

Here is the planned routine for the Chemistry Engineering student:

[
  {time:'7:00', location:'home', activity:'preparing for classes' },
  {time:'8:00', location:'FEQ', activity:'class' },
  {time:'12:00', location:'RU', activity:'lunch' },
  {time:'14:00', location:'FEQ', activity:'class' },
  {time:'17:00', location:'IMECC', activity:'class' },
  {time:'18:00', location:'RA', activity:'dinner' },
  {time:'20:00', location:'home', activity:'relax' }
]


In [78]:
# Latitude e longitude de locais de interesse
IC = (-22.813344, -47.063667)
FEEC = (-22.820788, -47.066489)

In [79]:
net = sumolib.net.readNet('osm.net.xml')
radius = 20
parkingAreas = list(sumolib.output.parse('osm_stops.add.xml', "parkingArea"))

In [7]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [8]:
def getClosestEdges(lat, lon, radius):
    # Gets the 10 closest edges to the given lat, lon
    x, y = net.convertLonLat2XY(lon, lat)
    edges = net.getNeighboringEdges(x, y, radius)
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])
        closestEdges = [x[1] for x in distanceAndEdges[:10]]
    
    return closestEdges

In [9]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        return parking_spot

    print("No parking spot found")
    return None

In [10]:
def getPath(coordinates, parkingAreas):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.
    path = []
    path.append(getClosestEdges(*coordinates[0], radius)[0].getID())
    for i in range(1, len(coordinates)-1):
        path.append(getParkingSpot(*coordinates[i], radius, parkingAreas))
    path.append(getClosestEdges(*coordinates[-1], radius)[0].getID())
    return path

In [56]:
def pathToXML(path, vehicleID, departure_time, stop_duration):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for stop in path[1:-1]:
        xml += f'\t<stop parkingArea="{stop}" duration="{stop_duration}"/>\n'
    xml += '</trip>'
    return xml

In [62]:
coordinates = [FEEC, IC, FEEC] # Essas coordenadas virão da LLM em algum momento
path = getPath(coordinates, parkingAreas)
xml = pathToXML(path, "vehicle1", 10, 100)
print(xml)

<trip id="vehicle1" depart="10" from="FEEC" to="FEEC">
	<stop parkingArea="pa75701374#0" duration="100"/>
</trip>


In [8]:
student_info = "Chemistry Engineering student. He has most classes at FEQ and has luch at RU and dinner at RA. Today is monday. The student starts and ends his day at home"
response = LLAMAconnect.getResponse_trip(student_info)
print(response)

Here is the planned routine for the Chemistry Engineering student:

[
  {time:'7:00', location:'home', activity:'preparing for classes' },
  {time:'8:00', location:'FEQ', activity:'class' },
  {time:'12:00', location:'RU', activity:'lunch' },
  {time:'14:00', location:'FEQ', activity:'class' },
  {time:'17:00', location:'IMECC', activity:'class' },
  {time:'18:00', location:'RA', activity:'dinner' },
  {time:'20:00', location:'home', activity:'relax' }
]


In [5]:
print(response.split("{")[2])

time:'8:00', location:'FEQ', activity:'class' },
  


In [55]:
# Trying to get response from LLAMA, but it's not working properly
response = LLAMAconnector.getResponse(path, "veh_1", 10, 100)
print(response)

<pre><trip id='veh_1' type='veh_passenger' depart='10' departLane='best' from='FEEC' to='FEEC'> 
   <stop parkingArea='pa75701374#0' duration='100' />
</trip></pre>
